In [43]:
#USED: September 29, 2020
from sklearn.model_selection import train_test_split as TTS, cross_validate as cv 
from sklearn.preprocessing import MinMaxScaler as MMS, StandardScaler as SScaler
from sklearn.metrics import mean_squared_error as MSE, r2_score as r2
from matplotlib import pyplot as plt
import time, scipy, numpy as np, math, pickle, random, warnings, pandas as pd, csv, hyperas
from sklearn.pipeline import Pipeline
from hyperopt import STATUS_OK, tpe, Trials, hp, fmin
from functools import partial
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers import Dense
from keras.models import Sequential
from tensorflow.python.keras import backend as K
from keras.optimizers import Adam
import tensorflow as tf
from tensorflow.python.framework import ops
import flair
from joblib import dump, load
import warnings
#'Ex1' : hp.uniform('Ex1', -8, 4),
#'Ex2' : hp.uniform('Ex2', -25, -5),
#'Ex3' : hp.uniform('Ex3', -6, 3),
#


    #This notebook contains algorithm for searching for io values that fit data from current and FE data
    #derived YOGI


#
#Last Used Nov 1
#Fundamentals of charge transfer on an electrode: JACS, 2005, 127, 7815-7824; JACS, 2005, 127, 13949-13954; J. Phys. Chem. B, 2006, 110, 19433-19442 (optional)

In [55]:
#declare your global variables here
scale = load('./Wire/1200_Results/Wire_Scale_1200.bin')
i_model = tf.keras.models.load_model('./Wire/1200_Results/i_total_Model')
fe_model = tf.keras.models.load_model('./Wire/1200_Results/FE_Model')
def gen_search_space(x):
    mod_ss = {
        'P' : hp.uniform('P', 1, 50),
        'L' : hp.uniform('L', x, 150), #5, 150
        'D' : hp.uniform('D', 1, 10),
    }
    return mod_ss
#{'Ex1': -5.08498, 'Ex2': -7.09325, 'Ex3': -2.37213, 'dif': 339.0, 'o1': 0.98413
lst = [[-0.6, 0.1, 20, 20, 5]] #CHANGE
lst = scale.transform(lst)
print(fe_model.predict(lst))
print(5*(0.79-0.00819)/0.00819)

[[0.8504277]]
477.2954822954824


In [56]:
def fix_indices(ar, ar2):
    #parameters passed in wrong order... annoying swap necessary... dicts are not inherently ordered
    ar2[0][2] = ar.get('P')
    ar2[0][3] = ar.get('L')
    ar2[0][4] = ar.get('D') 
    return ar2

In [57]:
def adj_potentials(pot, order):
    ar = np.array([pot,order,0,0,0]) #CHANGE
    ar = ar.reshape(1,-1)
    ar = scale.transform(ar)
    return (ar[0][0], ar[0][1])

def hypertune_BO(params, pot, order):
    lst = list(range(5)) #CHANGE
    ar2 = np.array(lst, ndmin=2, dtype=float)
    #parameters passed in wrong order... annoying swap necessary... dicts are not inherently ordered
    ar2 = fix_indices(params, ar2)
    ar2 = scale.transform(ar2) 
    ar2[0][0] = pot
    ar2[0][1] = order
    pred_i = i_model.predict(ar2)
    pred_fe = fe_model.predict(ar2)

    if (ar2[0][2] - ar2[0][4] < 1): pred_fe *= 1e-10
    #if (pred_fe < limit and lim_type == 'floor'):
     #   pred_fe *= 1e-10
    #elif (pred_fe > limit and lim_type == 'ceiling'):
     #   pred_fe *= 1e-10
    max_err = 1/(abs(pred_fe)*pred_i) #MAX
    
    return {'loss':max_err, 'params':params, 'max_loss':max_err, 'status':STATUS_OK} #'max_loss':max_err

In [65]:
def run_trials(pot, order, file_name, mod_ss):

    trials_step = 10  # how many additional trials to do after loading saved trials. 1 = save after iteration
    max_trials = 10  # initial max_trials. put something small to not have to wait
    #print(f'Trials Potentials {pots}')
    
    try:  # try to load an already saved trials object, and increase the max
        trials = pickle.load(open(file_name, 'rb')) #CHANGE
        max_trials = len(trials.trials) + trials_step
        #print("Rerunning from {} trials to {} (+{}) trials".format(len(trials.trials), max_trials, trials_step))
    except:  # create a new trials object and start searching
        trials = Trials()
        
    objective = partial(hypertune_BO, pot=pot, order=order) 
    best = fmin(fn=objective, space=mod_ss, algo=tpe.suggest, max_evals=max_trials, trials=trials)
    #CORRECTED ABOVE
    
    for key in best.keys(): best[key] = round(best.get(key),5)
    print("Best:", best)
    
    lst = [[0, 0, best.get('P'), best.get('L'), best.get('D')]]
    lst = scale.transform(lst)
    lst[0][0] = pot
    lst[0][1] = order
    FE = fe_model.predict(lst)[0][0]
    Cur = i_model.predict(lst)[0][0]
    print("FE : " + str(FE))
    #print("Real FE: " + str(data['FE']))
    print("Cur : " + str(Cur))
    #print("Real Cur: " + str(data['i']))
    # save the trials object
    with open(file_name, 'wb') as f: #CHANGE
        pickle.dump(trials, f)
    return trials, best

# loop indefinitely and stop whenever you like
#need to give the true vals to trials
def BO_Search(pot, order, file_name, mod_ss):
    i = 0
    iters = 1
    #print(f'BO_Search Potentials {pots}')
    fixed_cons = adj_potentials(pot, order)
    pot = fixed_cons[0]
    order = fixed_cons[1]
    while (i < iters): 
        trials, best = run_trials(pot, order, file_name, mod_ss)
        i+=1
    return best

In [69]:
pots = [-0.53, -0.62, -0.71, -0.82, -0.9, -1.01]
ords = [0.131, 0.084, 0.865, 0.844, 0.717, 0.8]
floors = [10*(i+1) for i in range(14)]
results = []
for x in floors:
    for i in range(len(pots)):
        pot = pots[i]
        order = ords[i]
        mod_ss = gen_search_space(x)
        file_name = './Wire/BO_Results/Cond_' + str(i) + '_' + str(x) + '.hyperopt'
        best = BO_Search(pot, order, file_name, mod_ss)
        print('\n\n\n\n\n' + file_name + ' ' + str(best) + '\n\n\n\n\n\n') 
        ar = [[pots[i], ords[i], best.get('P'), best.get('L'), best.get('D')]]
        ar = scale.transform(ar)
        fe = fe_model.predict(ar)
        cur = i_model.predict(ar)
        results.append([pots[i], ords[i], best.get('P'), best.get('L'), best.get('D'), x, fe[0][0], cur[0][0]])
df = pd.DataFrame(results, columns=['E', 'ord', 'P', 'L', 'D', 'Floor', 'FE', 'Cur'])
df.to_csv('Results_BO.csv', index=False)

100%|██████████| 3950/3950 [00:01<00:00,  7.86trial/s, best loss: 2.411872386932373]
Best: {'D': 5.88969, 'L': 149.95465, 'P': 46.00081}
FE : 0.6779665
Cur : 0.61155784





./Wire/BO_Results/Cond_0_10.hyperopt {'D': 5.88969, 'L': 149.95465, 'P': 46.00081}






100%|██████████| 3930/3930 [00:01<00:00,  7.34trial/s, best loss: 1.3374828100204468]
Best: {'D': 3.24076, 'L': 10.17713, 'P': 34.10952}
FE : 0.8984146
Cur : 0.8322142





./Wire/BO_Results/Cond_1_10.hyperopt {'D': 3.24076, 'L': 10.17713, 'P': 34.10952}






100%|██████████| 3930/3930 [00:01<00:00,  8.82trial/s, best loss: 0.983465313911438]
Best: {'D': 3.15214, 'L': 10.08902, 'P': 33.68904}
FE : 0.93338084
Cur : 1.0893865





./Wire/BO_Results/Cond_2_10.hyperopt {'D': 3.15214, 'L': 10.08902, 'P': 33.68904}






100%|██████████| 3930/3930 [00:01<00:00,  8.75trial/s, best loss: 0.889066755771637]
Best: {'D': 4.58849, 'L': 10.01646, 'P': 49.9844}
FE : 0.9019846
Cur : 1.2470003





./Wire/BO_Results/Cond_3_10.hyperopt {'D': 4






./Wire/BO_Results/Cond_1_60.hyperopt {'D': 5.46863, 'L': 60.04805, 'P': 44.11041}






100%|██████████| 3930/3930 [00:01<00:00,  8.38trial/s, best loss: 1.0267319679260254]
Best: {'D': 6.76938, 'L': 60.01333, 'P': 49.99216}
FE : 0.8941267
Cur : 1.0892909





./Wire/BO_Results/Cond_2_60.hyperopt {'D': 6.76938, 'L': 60.01333, 'P': 49.99216}






100%|██████████| 3930/3930 [00:01<00:00,  8.40trial/s, best loss: 0.9611508250236511]
Best: {'D': 3.15447, 'L': 60.0335, 'P': 35.9233}
FE : 0.858393
Cur : 1.2120547





./Wire/BO_Results/Cond_3_60.hyperopt {'D': 3.15447, 'L': 60.0335, 'P': 35.9233}






100%|██████████| 3930/3930 [00:01<00:00,  7.87trial/s, best loss: 1.0563597679138184]
Best: {'D': 1.00573, 'L': 60.05623, 'P': 24.03622}
FE : 0.7075037
Cur : 1.3380103





./Wire/BO_Results/Cond_4_60.hyperopt {'D': 1.00573, 'L': 60.05623, 'P': 24.03622}






100%|██████████| 3930/3930 [00:01<00:00,  8.30trial/s, best loss: 1.206368088722229]
Best: {'D': 5.31411, 'L': 148.1168, 'P': 43.

100%|██████████| 3930/3930 [00:01<00:00,  8.31trial/s, best loss: 0.994245707988739]
Best: {'D': 3.50217, 'L': 110.03361, 'P': 35.29764}
FE : 0.81892663
Cur : 1.2281778





./Wire/BO_Results/Cond_3_110.hyperopt {'D': 3.50217, 'L': 110.03361, 'P': 35.29764}






100%|██████████| 3930/3930 [00:01<00:00,  8.32trial/s, best loss: 1.1000670194625854]
Best: {'D': 1.79183, 'L': 141.48206, 'P': 28.21981}
FE : 0.6572371
Cur : 1.3831166





./Wire/BO_Results/Cond_4_110.hyperopt {'D': 1.79183, 'L': 141.48206, 'P': 28.21981}






100%|██████████| 3930/3930 [00:01<00:00,  8.32trial/s, best loss: 1.2064330577850342]
Best: {'D': 5.31579, 'L': 147.54911, 'P': 43.40817}
FE : 0.5559099
Cur : 1.4910504





./Wire/BO_Results/Cond_5_110.hyperopt {'D': 5.31579, 'L': 147.54911, 'P': 43.40817}






100%|██████████| 3930/3930 [00:01<00:00,  8.26trial/s, best loss: 2.4112846851348877]
Best: {'D': 5.99406, 'L': 149.98527, 'P': 46.44627}
FE : 0.67775005
Cur : 0.6119021





./Wire/BO_Results/Cond_0_120.hype

In [64]:
ar = [[-0.5, 0.1, 10, 10, 2]]
ar = scale.transform(ar)
fe_model.predict(ar)

array([[0.6075808]], dtype=float32)

In [ ]:
for i in range(len(data)):
    row = data.loc[i]
    old_source = source
    if(row['source'] != source):
        print(file_name)
        mod_ss = gen_search_space(len(pots))
        source = row['source']
        pots = [row['E']]
        fes = [round(row['FE'],3)]
        curs = [round(row['i'],3)]
        file_name = 'Dec3_' + str(row['source']) + '.hyperopt'
        #best = BO_Search(pots, pots.copy(), fes, curs, file_name, mod_ss)
        #print('\n\n\n\n\n' + file_name + ' ' + str(best) + '\n\n\n\n\n\n') 
        #results.append(best)
    else:
        pots.append(row['E'])
        fes.append(round(row['FE'],3))
        curs.append(round(row['i'],3))
    i += 1
#with open('./ogHERandCER_Model/BO_results_Dec20.pkl', 'wb') as f: #CHANGE
#    pickle.dump(results, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('./Sep15/BO_Results.pkl', 'wb') as f: #CHANGE
    pickle.dump(results, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
import pickle

with open('./Sep15/BO_Results.pkl', 'rb') as f:
    t = pick.load(f)

In [ ]:
objective = partial(hypertune_BO, model=key, scale=True) 
#max_cell_loss = 10000
#min_lss = 10000
#min_ind = 0
#end_iter_lss = 10000
#if (i >= iters - 1):
    #    results = trials.results[min_ind:]
    #   for j in range(len(trials.results) - min_ind):
    #        ind = j + min_ind
    #        if (ind >= len(trials.results)): break
    #        if ((trials.results[ind].get('max_loss') < max_cell_loss)): #& (trials.results[ind].get('loss') <= end_iter_lss)):
    #            min_lss = trials.results[ind].get('loss')
    #            max_cell_loss = trials.results[ind].get('max_loss')
    #            min_ind = ind
    #            if (i == iters - 1): end_iter_lss = min_lss
    #    print('Min Loss: ' + str(min_lss) + '\n' + 'Max Cell Loss: ' + str(max_cell_loss) + '\n' + 'End iter loss: ' + str(end_iter_lss) + '\n' + str(i) + '\n'')

In [ ]:
mod_ss = {
    'i1c' : hp.uniform('i1c', -8.5, 4),
    'i1h' : hp.uniform('i1h', -25, -5),
    'i2h' : hp.uniform('i2h', -6, 3),
    'dif' : hp.quniform('dif', 30, 350,1),
    'ord1' : hp.uniform('o1', 0, 1),
    'ord2' : hp.uniform('o2', 0, 1),
    'ord3' : hp.uniform('o3', 0, 1),
}



In [ ]:
ords = {}

for i in range(4):
    key = 'ord' + str(i+1)
    ords[key] = hp.uniform('o' + str(i+1), 0, 1)
print(ords)

In [ ]:
dic = {'o1':1, 'o2':2, 'o3':3}
ords = [dic.get('o' + str(i+1)) for i in range(3)]
ords


In [ ]:
hel = [0,1,2]
#hev = hel.copy()

print(hev)

In [ ]:
Yogi
'i1c' : hp.uniform('i1c', -8.5, 4),
'i1h' : hp.uniform('i1h', -25, -5),
'i2h' : hp.uniform('i2h', -6, 3),
'dif' : hp.quniform('dif', 30, 350,1),
    
Simple
'i1c' : hp.uniform('i1c', -15, 0),
'i1h' : hp.uniform('i1h', -17.3, -0.74),
'dif' : hp.quniform('dif', 30, 350,1),
    
og_HER
        'i1c' : hp.uniform('i1c', -15, 0),
        'i1h' : hp.uniform('i1h', -17.3, 0),
        'i2h' : hp.uniform('i2h', -25, 0),
        'dif' : hp.quniform('dif', 30, 350,1),